1. Downgrade Python version on Colab to ensure compatibility with the required dependencies

In [ ]:
# Downgrade Python by reinstalling pip and distutils
!apt-get install python3.8 python3-pip python3.8-distutils
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 1

# Check the result
!python --version

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.3).
python3.8 is already the newest version (3.8.18-1+jammy1).
python3.8-distutils is already the newest version (3.8.18-1+jammy1).
The following packages were automatically installed and are no longer required:
  blt distro-info-data gir1.2-glib-2.0 gir1.2-packagekitglib-1.0 libaio1
  libappstream4 libboost-atomic-dev libboost-atomic1.74-dev
  libboost-atomic1.74.0 libboost-chrono-dev libboost-chrono1.74-dev
  libboost-chrono1.74.0 libboost-container-dev libboost-container1.74-dev
  libboost-container1.74.0 libboost-context-dev libboost-context1.74-dev
  libboost-context1.74.0 libboost-coroutine-dev libboost-coroutine1.74-dev
  libboost-coroutine1.74.0 libboost-date-time-dev libboost-date-time1.74-dev
  libboost-date-time1.74.0 libboost-exception-dev libboost-exception1.74-dev
  libboost-fiber-dev libboost-fiber1.74-dev libboo

2. Install the required dependencies

In [ ]:
# import requirement.txt into colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# various dependencies
!pip3 install -r requirements.txt

# widgets for video playback, will move into requirements.txt
!pip3 install ipywidgets

  Using cached accelerate-0.16.0-py3-none-any.whl (199 kB)
  Using cached aiofiles-23.1.0-py3-none-any.whl (14 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... done
  Using cached anyio-3.6.2-py3-none-any.whl (80 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached av-10.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.0 MB)
  Using cached bitsandbytes-0.35.4-py3-none-any.whl (62.5 MB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached charset_normalizer-3.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (198 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached contourpy-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached decord-0.6.0-py3-none-manylinux20

3. Download a sample video and upload it

In [1]:
!pip install gdown

import gdown

file_id = '1j-MtW4Omn2n5zKFjojDGugaUg3dhnBKF'
output_file = 'video.mp4'
download_url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(download_url, output_file, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1j-MtW4Omn2n5zKFjojDGugaUg3dhnBKF
To: /content/video.mp4
100%|██████████| 13.9M/13.9M [00:00<00:00, 18.4MB/s]


'video.mp4'

4. Select a video and play it

In [2]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import os

video_folder = '/content'
video_options = [filename for filename in os.listdir(video_folder) if filename.endswith('.mp4')] # just testing for now
dropdown = widgets.Dropdown(
    options=video_options,
    description='Select Video:'
)

output = widgets.Output()
confirm_button = widgets.Button(
    description='Confirm Video',
    button_style='success'
)

def display_message(message, message_type='info'):
    with output:
        clear_output(wait=True)
        if message_type == 'error':
            display(HTML(f'<font color="red">{message}</font>'))
        else:
            display(HTML(message))

def is_video_playable(selected_video):
    video_path = os.path.join(video_folder, selected_video)
    return os.path.isfile(video_path)

def on_dropdown_change(change):
    selected_video = change['new']
    if selected_video:
        confirm_button.disabled = False
    else:
        confirm_button.disabled = True

def on_confirm_button_click(button):
    selected_video = dropdown.value
    if is_video_playable(selected_video):
        video_path = os.path.join(video_folder, selected_video)
        video_html = f'<video controls src="{video_path}" width="500" height="400" />'
        display_message(video_html)
    else:
        display_message("Selected video is not playable.", message_type='error')

dropdown.observe(on_dropdown_change, names='value')
confirm_button.on_click(on_confirm_button_click)

display(dropdown, confirm_button, output)


Dropdown(description='Select Video:', options=('video.mp4',), value='video.mp4')

Button(button_style='success', description='Confirm Video', style=ButtonStyle())

Output()